In [ ]:
from pathlib import Path

# filename = "Binary classification of CIFAR10 dataset<br>with resolution of (16x16x3) pixels"
# filename = "Binary classification of CIFAR10 dataset<br>with resolution of (32x32x3) pixels"
# filename = "Binary classification of FashionMNIST dataset<br>with resolution of (16x16) pixels"
# filename = "Binary classification of FashionMNIST dataset<br>with resolution of (28x28) pixels"
# filename = "Binary classification of MNIST dataset<br>with resolution of (16x16) pixels"
# filename = "Binary classification of MNIST dataset<br>with resolution of (28x28) pixels"

# filename = "Classification accuracy of CIFAR10 dataset<br>with resolution of (16x16x3) pixels"
# filename = "Classification accuracy of CIFAR10 dataset<br>with resolution of (32x32x3) pixels"
# filename = "Classification accuracy of FashionMNIST dataset<br>with resolution of (16x16) pixels"
filename = "Classification accuracy of FashionMNIST dataset<br>with resolution of (28x28) pixels"
# filename = "Classification accuracy of MNIST dataset<br>with resolution of (16x16) pixels"
# filename = "Classification accuracy of MNIST dataset<br>with resolution of (28x28) pixels"

# filename = "Classification accuracy of CNN<br>configs on the (28x28) FashionMNIST dataset"
# filename = "Classification accuracy of QCNN<br>configs on the (28x28) FashionMNIST dataset"
# filename = "Classification accuracy of MQCC<br>configs on the (28x28) FashionMNIST dataset"
# filename = "Classification accuracy of MQCC (Avg Pooling)<br>configs on the (16x16) FashionMNIST dataset"
# filename = "Classification accuracy of MQCC (Avg Pooling)<br>configs on the (28x28) FashionMNIST dataset"
# filename = "Classification accuracy of MQCC Optimized<br>configs on the (16x16) FashionMNIST dataset"
# filename = "Classification accuracy of Quanvolution<br>configs on the (16x16) FashionMNIST dataset"

# filename = "Number of training parameters for ML models<br> for tested data sizes"
# filename = "Circuit depth of variational quantum classifiers<br> for tested data sizes"
# filename = "Gate count of variational quantum classifiers<br> for tested data sizes"

path = Path(f"/Users/mingyoungjeng/Downloads/figures_thesis/{filename}")

glob = "**/*.csv"


In [ ]:
if "Binary classification of" in filename:
    chart_type = "loss"
if "accuracy" in filename:
    chart_type = "accuracy"
if "tested data sizes" in filename:
    chart_type = "bar"
    
if "CIFAR10" in filename:
    dataset = "CIFAR10"
if "MNIST" in filename:
    dataset = "MNIST"
if "FashionMNIST" in filename:
    dataset = "FashionMNIST"
    
# chart_type, dataset


In [ ]:
from fnmatch import fnmatch
from polars import DataFrame, read_csv

# if path.is_dir():
#     dfs = [read_csv(file) for file in path.glob(glob)]
# else:
#     if fnmatch(path, glob):
#         df = read_csv(path)

dfs = tuple(read_csv(file) for file in path.glob(glob))
kwargs = [dict(name=file.stem) for file in path.glob(glob)]
title = path.stem


In [ ]:
for i, kwarg in enumerate(kwargs):
    name = kwarg.get("name", "")
    if "mlp" in name:
        kwarg["name"] = "MLP"
        kwarg["marker_color"] = "#5e00b0" # purple
        kwarg["marker_symbol"] = "square"
        # kwarg["line_dash"] = "solid"
        
    if "cnn" in name:
        kwarg["name"] = "CNN"
        kwarg["marker_color"] = "#0069B1" # blue
        kwarg["marker_symbol"] = "square"
        # kwarg["line_dash"] = "solid"
    
    if "mqcc" in name:
        kwarg["name"] = "MQCC"
        kwarg["marker_color"] = "#ff0d00" # red
        # kwarg["marker_symbol"] = "circle"
        # kwarg["line_dash"] = "solid"
    
    if "quanvolution" in name:
        kwarg["name"] = "Quanvolution"
        kwarg["marker_color"] = "#e88f09" # orange
        kwarg["marker_symbol"] = "diamond"
        # kwarg["line_dash"] = "solid"
    
    if ("qcnn" in name) | ("baseline" in name):
        kwarg["name"] = "QCNN"
        kwarg["marker_color"] = "#45b02a" # green
        kwarg["marker_symbol"] = "diamond"
        # kwarg["line_dash"] = "solid"
        
    if ("mqcc-optimized" in name) | ("mqcc_optimized" in name) | ("mqcc optimized" in name):
        kwarg["name"] = "MQCC Optimized"
        kwarg["marker_color"] = "#AF231C" # dark red
        # kwarg["marker_symbol"] = "circle"
        # kwarg["line_dash"] = "solid"
    
    if "tested data sizes" in filename:
        kwarg["type"] = "bar"


In [ ]:
import polars as pl

def rolling_mean(df, window_size):
    return df.select(pl.all().rolling_mean(window_size, center=True)).drop_nulls()

if chart_type == "loss":
    dfs = (rolling_mean(df, 40) for df in dfs)


In [ ]:
from qcc.graph import process_dataframe

dfs = [process_dataframe(df, as_plotly=True) if len(df.columns) > 1 else dict(y=list(df[df.columns[0]])) for df in dfs]

print(dfs[0])

for df, kwarg in zip(dfs, kwargs):
    df.update(kwarg)
    
    if chart_type != "bar":
        df.update(x=list(range(1, len(df["y"])+1)))
    
    # if "error_y" in df:
    #     del df["error_y"]


In [ ]:
from qcc.graph import plot

dfs[:] = sorted(dfs, key=lambda x: x.get("name", ""))

# if "variational quantum classifiers" in title:
#     dfs[:] = [dfs[i] for i in [2, 0, 1]]
# else:
#     dfs[:] = [dfs[i] for i in [1, 0, 5, 4, 2, 3]]
    
kwargs = dict(title=title)

# Log Loss
if chart_type == "loss":
    kwargs.update(ylabel="Log Loss", xlabel="Iteration", xaxis_tickformat= ',.2r', yaxis_title_standoff=6)

    if dataset == "CIFAR10":
        kwargs.update(yaxis_range = [0.0, 0.8], legend=(0.05, 0.4), xaxis_dtick=1000) # CIFAR10
    if dataset == "FashionMNIST":
        kwargs.update(yaxis_range = [0, 0.8], legend=(0.60, 0.65), xaxis_dtick=2000) # FashionMNIST
    if dataset == "MNIST":
        kwargs.update(yaxis_range = [0, 0.8], legend=(0.60, 0.55), xaxis_dtick=2000) # MNIST

# Accuracy
if chart_type == "accuracy":
    kwargs.update(ylabel="Accuracy", xlabel="Epoch", yaxis_tickformat= ',.0%', xaxis_dtick=1)
    if dataset == "CIFAR10":
        kwargs.update(yaxis_range = [0.45, 0.9], legend=(0.6, 0.4), yaxis_dtick=0.1) # CIFAR10
    if dataset == "FashionMNIST":
        kwargs.update(yaxis_range = [0.85, 1], legend=(0.6, 0.45), yaxis_dtick=0.02) # FashionMNIST
    if dataset == "MNIST":
        kwargs.update(yaxis_range = [0.91, 1], legend=(0.6, 0.45), yaxis_dtick=0.01) # MNIST

# Bar chart
if chart_type == "bar":
    for df in dfs:
        if "marker_symbol" in df:
            del df["marker_symbol"]
    
    if "depth" in title:
        kwargs.update(ylabel="Circuit depth", xlabel="Data Size", legend=(0.05, 0.95), yaxis_range = [0, 1200])
    if "Gate count" in title:
        kwargs.update(ylabel="Gate count", xlabel="Data Size", legend=(0.05, 0.95), yaxis_range = [0, 1200])
    if "parameters" in title:
        kwargs.update(ylabel="Number of training parameters", xlabel="Data Size", legend=(0.05, 0.95), yaxis_title_standoff=0, yaxis_type="log", yaxis_dtick=1, yaxis_range = [1, 11])


In [ ]:
import numpy as np

# dfs[:] = sorted(dfs, key=lambda x: x.get("y", "")[-1], reverse=True)
# kwargs.update(yaxis_range = [0.72, 1], legend=(0.5, 0.65), yaxis_dtick=0.05)

# dfs[:] = [dfs[i] for i in [1, 0]]
# kwargs.update(yaxis_range = [0.89, 0.95], legend=(0.6, 0.2), yaxis_dtick=0.01,)

# dfs[:] = sorted(dfs, key=lambda x: x.get("y", "")[-1], reverse=True)
# kwargs.update(yaxis_range = [0.72, 0.98], legend=(0.45, 0.6), yaxis_dtick=0.02, legend_font_size=10)

# dfs[:] = sorted(dfs, key=lambda x: x.get("y", "")[-1], reverse=True)
# kwargs.update(yaxis_range = [0.84, 0.98], legend=(0.44, 0.6), yaxis_dtick=0.02, legend_font_size=10)

# dfs[:] = sorted(dfs, key=lambda x: np.max(x.get("y", "")), reverse=True)
# kwargs.update(yaxis_range = [0.6, 0.98], legend=(0.45, 0.6), yaxis_dtick=0.04, legend_font_size=10)

# dfs[:] = [dfs[i] for i in [1, 0]]
# kwargs.update(yaxis_range = [0.94, 0.99], legend=(0.5, 0.4), yaxis_dtick=0.01)


In [ ]:
fig = plot(*dfs, **kwargs)
fig.show()


In [ ]:
fig.write_image(f"img/{filename}.png", scale=3)


In [ ]:
# from pathlib import Path
# import plotly.io as pio

# path = Path("/Users/mingyoungjeng/Downloads/figures_thesis")
# for file in path.glob("*.json"):
#     fig = pio.read_json(file)
#     fig.write_image(file.with_suffix(.png), scale=3)
